In [ ]:
import pathlib
from pathlib import Path
import pandas as pd

import shutil

In [2]:
!ls

3rdparty		include
backup			json_mjpeg_streams.sh
build			LICENSE
build.ps1		Makefile
cfg			net_cam_v3.sh
cmake			net_cam_v4.sh
CMakeLists.txt		obj
coco_output		README.md
darknet			results
DarknetConfig.cmake.in	scripts
darknet_images.py	src
darknet.py		test_remote_1.ipynb
darknet_video.py	valladolid_merida_8clases_25Oct2022
data			vcpkg.json
image_yolov3.sh		video_yolov3.sh
image_yolov4.sh		video_yolov4.sh


In [3]:
!ls valladolid_merida_8clases_25Oct2022

img  obj_8clases.names


In [4]:
print(Path.cwd())

/home/jose/darknet


In [5]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Jun__8_16:49:14_PDT_2022
Cuda compilation tools, release 11.7, V11.7.99
Build cuda_11.7.r11.7/compiler.31442593_0
torch:  1.13 ; cuda:  1.13.0
detectron2: 0.6


In [6]:
!fiftyone convert -h

usage: fiftyone convert [-h] --input-type INPUT_TYPE --output-type OUTPUT_TYPE
                        [--input-dir INPUT_DIR]
                        [--input-kwargs KEY=VAL [KEY=VAL ...]]
                        [--output-dir OUTPUT_DIR]
                        [--output-kwargs KEY=VAL [KEY=VAL ...]] [-o]

Convert datasets on disk between supported formats.

    Examples::

        # Convert an image classification directory tree to TFRecords format
        fiftyone convert \
            --input-dir /path/to/image-classification-directory-tree \
            --input-type fiftyone.types.ImageClassificationDirectoryTree \
            --output-dir /path/for/tf-image-classification-dataset \
            --output-type fiftyone.types.TFImageClassificationDataset

        # Convert a COCO detection dataset to CVAT image format
        fiftyone convert \
            --input-dir /path/to/coco-detection-dataset \
            --input-type fiftyone.types.COCODetectionDataset \
            --outpu

In [11]:
INPUT_DIR = str(Path.cwd() / "valladolid_merida_8clases_25Oct2022")
OUTPUT_DIR = str(Path.cwd() / "coco_output")

#INPUT_DIR = str("/valladolid_merida_8clases_25Oct2022")
#OUTPUT_DIR = str("/coco_output")

In [15]:
INPUT_DIR

'/home/jose/darknet/valladolid_merida_8clases_25Oct2022'

In [16]:
!fiftyone convert \
    --input-dir $INPUT_DIR --input-type fiftyone.types.YOLOv4Dataset \
    --output-dir $OUTPUT_DIR --output-type fiftyone.types.COCODetectionDataset

Using input format: fiftyone.types.dataset_types.YOLOv4Dataset
Using export format: fiftyone.types.dataset_types.COCODetectionDataset
Loading dataset from '/home/jose/darknet/valladolid_merida_8clases_25Oct2022'
Images file '/home/jose/darknet/valladolid_merida_8clases_25Oct2022/images.txt' not found. Listing data directory '/home/jose/darknet/valladolid_merida_8clases_25Oct2022/data/' instead
 100% |█████| 2419/2419 [13.1s elapsed, 0s remaining, 170.9 samples/s]      
Exporting dataset to '/home/jose/darknet/coco_output'
Directory '/home/jose/darknet/coco_output' already exists; export will be merged with existing files
Directory '/home/jose/darknet/coco_output/data/' already exists; export will be merged with existing files
 100% |█████| 2419/2419 [6.1m elapsed, 0s remaining, 3.1 samples/s]       


In [17]:
# separar en 3 carpetas. train, val, test


In [18]:
folder_path = Path.cwd() / "valladolid_merida_8clases_25Oct2022" / "data"

In [19]:
folder_path

PosixPath('/home/jose/darknet/valladolid_merida_8clases_25Oct2022/data')

In [20]:
# leyendo la lista de los nombres y archivos
all_images = [img.name for img in folder_path.glob('*.png')]
all_txts = [txt.name for txt in folder_path.glob('*.txt')]
all_names = [img.stem for img in folder_path.glob('*.png')]

In [23]:
# guardando las listas
pd.Series(all_images).to_csv(Path.cwd()/"valladolid_merida_8clases_25Oct2022"/"imgs.txt",header=None,sep=" ",index=False)
pd.Series(all_txts).to_csv(Path.cwd()/"valladolid_merida_8clases_25Oct2022"/"txts.txt",header=None,sep=" ",index=False)
pd.Series(all_names).to_csv(Path.cwd()/"valladolid_merida_8clases_25Oct2022"/"names.txt",header=None,sep=" ",index=False)

In [24]:
the_names = pd.read_csv(Path.cwd()/"valladolid_merida_8clases_25Oct2022"/"names.txt",header=None,sep=" ").iloc[:,0].to_list()

In [26]:
# train 80% test 10% val 10%

from sklearn.model_selection import train_test_split
chosen_train, chosen_rest = train_test_split(the_names, test_size=0.20, random_state=42)
chosen_test, chosen_val = train_test_split(chosen_rest, test_size=0.50, random_state=42)

In [27]:
original_folder = "valladolid_merida_8clases_25Oct2022"

In [28]:
pd.Series(chosen_train).to_csv(Path.cwd()/original_folder/"chosen_train.txt",header=None,sep=" ",index=False)
pd.Series(chosen_test).to_csv(Path.cwd()/original_folder/"chosen_test.txt",header=None,sep=" ",index=False)
pd.Series(chosen_val).to_csv(Path.cwd()/original_folder/"chosen_val.txt",header=None,sep=" ",index=False)

In [35]:
def copier(new_f,c):
    
    temp = Path.cwd() / original_folder / new_f 
    
    temp.mkdir(parents=True, exist_ok=True)
    
    for file in c:
    
        img_path = folder_path / (file + ".png")
        txt_path = folder_path / (file + ".txt")

        new_img_path = temp / (file + ".png")
        new_txt_path = temp / (file + ".txt")

        shutil.copy2(str(img_path), str(new_img_path))
        shutil.copy2(str(txt_path), str(new_txt_path))
        
    print(new_f, "folder is done")


In [36]:
copier("train",chosen_train)

train folder is done


In [37]:
copier("test",chosen_test)

test folder is done


In [38]:
copier("val",chosen_val)

val folder is done


In [39]:
# convertir a formato COCO

In [49]:
def coco_doer(purpose):
    
    INPUT_DIR = str(Path.cwd() / original_folder / purpose )
    
    tmp_str = "coco_" + purpose
    OUTPUT_DIR = Path.cwd() / tmp_str
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    OUTPUT_DIR = str(OUTPUT_DIR)

    !fiftyone convert \
        --input-dir $INPUT_DIR --input-type fiftyone.types.YOLOv4Dataset \
        --output-dir $OUTPUT_DIR --output-type fiftyone.types.COCODetectionDataset

In [50]:
# tuve que agregar un nivel las carpetas, renombrar a data y agregar el obj.names

In [51]:
coco_doer("train")

Using input format: fiftyone.types.dataset_types.YOLOv4Dataset
Using export format: fiftyone.types.dataset_types.COCODetectionDataset
Loading dataset from '/home/jose/darknet/valladolid_merida_8clases_25Oct2022/train'
Images file '/home/jose/darknet/valladolid_merida_8clases_25Oct2022/train/images.txt' not found. Listing data directory '/home/jose/darknet/valladolid_merida_8clases_25Oct2022/train/data/' instead
 100% |█████| 1935/1935 [11.3s elapsed, 0s remaining, 159.1 samples/s]      
Exporting dataset to '/home/jose/darknet/coco_train'
Directory '/home/jose/darknet/coco_train' already exists; export will be merged with existing files
Directory '/home/jose/darknet/coco_train/data/' already exists; export will be merged with existing files
 100% |█████| 1935/1935 [4.8m elapsed, 0s remaining, 12.4 samples/s]      


In [52]:
coco_doer("test")

Using input format: fiftyone.types.dataset_types.YOLOv4Dataset
Using export format: fiftyone.types.dataset_types.COCODetectionDataset
Loading dataset from '/home/jose/darknet/valladolid_merida_8clases_25Oct2022/test'
Images file '/home/jose/darknet/valladolid_merida_8clases_25Oct2022/test/images.txt' not found. Listing data directory '/home/jose/darknet/valladolid_merida_8clases_25Oct2022/test/data/' instead
 100% |███████| 242/242 [1.5s elapsed, 0s remaining, 159.0 samples/s]         
Exporting dataset to '/home/jose/darknet/coco_test'
Directory '/home/jose/darknet/coco_test' already exists; export will be merged with existing files
 100% |███████| 242/242 [19.4s elapsed, 0s remaining, 12.2 samples/s]      


In [53]:
coco_doer("val")

Using input format: fiftyone.types.dataset_types.YOLOv4Dataset
Using export format: fiftyone.types.dataset_types.COCODetectionDataset
Loading dataset from '/home/jose/darknet/valladolid_merida_8clases_25Oct2022/val'
Images file '/home/jose/darknet/valladolid_merida_8clases_25Oct2022/val/images.txt' not found. Listing data directory '/home/jose/darknet/valladolid_merida_8clases_25Oct2022/val/data/' instead
 100% |███████| 242/242 [1.4s elapsed, 0s remaining, 176.5 samples/s]         
Exporting dataset to '/home/jose/darknet/coco_val'
Directory '/home/jose/darknet/coco_val' already exists; export will be merged with existing files
 100% |███████| 242/242 [24.0s elapsed, 0s remaining, 12.9 samples/s]      
